In [4]:
import os

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import torch.nn as nn
import numpy as np
import torchaudio
import torch


In [5]:
def audio_to_spectrogram(file_path):
    waveform, sr = torchaudio.load(file_path)

    transformer = torchaudio.transforms.MelSpectrogram(
        sample_rate=sr, n_fft=2048, hop_length=512, n_mels=64
        )

    spectrogram = transformer(waveform)
    spectrogram = torchaudio.transforms.AmplitudeToDB()(spectrogram)
    return spectrogram.squeeze(0).transpose(0, 1)  # Transpose to have time steps on the first dimension


def vector_quantize(features, n_clusters=100):
    kmeans = KMeans(n_clusters=n_clusters)
    all_data = np.vstack([f.numpy() for f in features])
    kmeans.fit(all_data)

    quantized_features = [
        torch.tensor(kmeans.predict(f.numpy()), dtype=torch.long)
        for f in features
        ]

    return quantized_features, kmeans


def load_and_quantize_data(
        directory,
        target_labels=["up", "down", "left", "right"],
        n_clusters=100
        ):

    features = []
    labels = []
    for label in os.listdir(directory):
        if label in target_labels:
            class_dir = os.path.join(directory, label)
            for fname in os.listdir(class_dir):
                file_path = os.path.join(class_dir, fname)
                spectrogram = audio_to_spectrogram(file_path)
                features.append(spectrogram)
                labels.append(label)

    quantized_features, _ = vector_quantize(features, n_clusters)

    return quantized_features, labels


def pad_sequences(sequences, pad_value=0):
    # Oblicz maksymalną długość sekwencji
    max_len = max([s.size(0) for s in sequences])

    # Wypełnij każdy tensor w liście, aby miał maksymalną długość
    padded_sequences = [
        torch.nn.functional.pad(s, (0, max_len - s.size(0)), value=pad_value)
        for s in sequences
        ]

    # Stos wszystkich wypełnionych tensorów
    return torch.stack(padded_sequences)


class AudioTransformer(nn.Module):
    def __init__(
            self,
            num_tokens, dim_model, num_heads, num_classes,
            dim_feedforward=2048, num_layers=1, dropout=0.1
            ):

        super().__init__()
        self.embedding = nn.Embedding(num_tokens, dim_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim_model, nhead=num_heads,
            dim_feedforward=dim_feedforward, dropout=dropout
            )

        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer, num_layers=num_layers
            )

        self.fc = nn.Linear(dim_model, num_classes)

    def forward(self, src):
        src = self.embedding(src)  # Replace tokens with embeddings
        output = self.transformer_encoder(src)
        output = output.mean(dim=1)  # Average over the sequence for classification
        output = self.fc(output)
        return output


In [6]:
# Load data and vector quantize
features, labels = load_and_quantize_data("data/train")
# test_features, test_labels = load_and_quantize_data("data/test")


In [7]:
# Użycie funkcji pad_sequences
features_padded = pad_sequences(features)

train_features_padded, test_features_padded, train_labels, test_labels = train_test_split(
    features_padded, labels, test_size=0.2, random_state=42, stratify=labels
)

# Convert to tensor and encode labels
label_encoder = LabelEncoder()
train_labels_encoded = torch.tensor(label_encoder.fit_transform(train_labels))
test_labels_encoded = torch.tensor(label_encoder.transform(test_labels))

# Create datasets
train_dataset = TensorDataset(train_features_padded, train_labels_encoded)
test_dataset = TensorDataset(test_features_padded, test_labels_encoded)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [8]:
# Model
num_tokens = 100  # Same as number of clusters
model = AudioTransformer(
    num_tokens=num_tokens,
    dim_model=256,
    num_heads=8,
    num_classes=len(np.unique(train_labels))
    )

# Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(10):
    loss_sum = []
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        loss_sum.append(loss.item())

    print(f"Epoch {epoch+1}, Loss: {np.mean(np.array(loss_sum))}")


c:\Users\1234o\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\1234o\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1, Loss: 1.248584811091423
Epoch 2, Loss: 1.0252215158939362
Epoch 3, Loss: 0.9800149464607238
Epoch 4, Loss: 0.9249011129140854
Epoch 5, Loss: 0.9184495961666107
Epoch 6, Loss: 0.9024950790405274
Epoch 7, Loss: 0.8926821839809418
Epoch 8, Loss: 0.880125060081482
Epoch 9, Loss: 0.8714895620942116
Epoch 10, Loss: 0.8625024443864823


In [9]:
model.eval()  # Przełącz model w tryb oceny
test_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        labels = labels.long()  # Upewnij się, że etykiety są typu long
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader.dataset)
test_accuracy = 100 * correct / total
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


Test Loss: 0.9864, Test Accuracy: 60.25%


In [12]:
# from datasets import load_dataset

# minds = load_dataset("PolyAI/minds14", name="en-AU", split="train")

c:\Users\1234o\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 654 examples [00:00, 6519.81 examples/s]


In [13]:
# minds

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 654
})

In [2]:
from transformers import pipeline

classifier = pipeline(
    "audio-classification",
    model="anton-l/xtreme_s_xlsr_300m_minds14",
)



Some weights of the model checkpoint at anton-l/xtreme_s_xlsr_300m_minds14 were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at anton-l/xtreme_s_xlsr_300m_minds14 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos

In [7]:
classifier_commands = pipeline(
    "audio-classification", model="MIT/ast-finetuned-speech-commands-v2"
)

c:\Users\1234o\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\1234o\.cache\huggingface\hub\models--MIT--ast-finetuned-speech-commands-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Error while downloading from https://cdn-lfs.huggingface.co/repos/60/d7/6

ValueError: Could not load model MIT/ast-finetuned-speech-commands-v2 with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForAudioClassification'>, <class 'transformers.models.audio_spectrogram_transformer.modeling_audio_spectrogram_transformer.ASTForAudioClassification'>). See the original errors:

while loading with AutoModelForAudioClassification, an error is thrown:
Traceback (most recent call last):
  File "c:\Users\1234o\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\response.py", line 737, in _error_catcher
    yield
  File "c:\Users\1234o\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\response.py", line 862, in _raw_read
    data = self._fp_read(amt, read1=read1) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\1234o\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\response.py", line 845, in _fp_read
    return self._fp.read(amt) if amt is not None